# Task 1:  Polyp Segmentation Task (Training)

In [1]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib import image
from numpy import savez_compressed
from torchvision import transforms
import copy
import random


import torch
from gan import Generator, Discriminator  # models are in gan.py file
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK']='True'





In [2]:
def match_im_msk(img_files,msk_files):
    
    pure_img_names=[]
    pure_msk_names=[]

    for im in img_files:
        pure_img_names+= [os.path.basename(im)]
    for msk in msk_files:
        pure_msk_names+= [os.path.basename(msk)]
    
    return(pure_img_names,pure_msk_names)

In [3]:
# loading data from the resized set
data_dir = "./data/task1/resized/train/"
img_files = glob.glob(data_dir+"images/*.jpg")
msk_files=glob.glob(data_dir+"masks/*.jpg")
pure_img_names,pure_msk_names=match_im_msk(img_files,msk_files)
pure_img_names==pure_msk_names

True

In [4]:
#normalizing the image data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std =[0.5, 0.5, 0.5],
    ),
])

In [5]:


loaded_images = list()
loaded_masks = list()

for i,filename in enumerate(img_files):
    # load image
    img_data = image.imread(filename)
    # store loaded image
    img_data_tr=transform(img_data.copy()).numpy()
    loaded_images.append(img_data_tr)
    if i%500 ==0:
        print('> loaded image %s %s' % (filename, img_data_tr.shape))
for i,filename in enumerate(msk_files):
    # load image
    msk_data = image.imread(filename)
    msk_data_tr=transform(msk_data.copy()).numpy()
    # store loaded image
    loaded_masks.append(msk_data_tr)
    if i%500 ==0:
        print('> loaded mask %s %s' % (filename, msk_data_tr.shape))


> loaded image ./data/task1/resized/train/images\cju0qkwl35piu0993l0dewei2.jpg (3, 512, 512)
> loaded image ./data/task1/resized/train/images\cju5bdwa3aatx0818b79i18zf.jpg (3, 512, 512)
> loaded mask ./data/task1/resized/train/masks\cju0qkwl35piu0993l0dewei2.jpg (3, 512, 512)
> loaded mask ./data/task1/resized/train/masks\cju5bdwa3aatx0818b79i18zf.jpg (3, 512, 512)


In [6]:
# we can show some samples
# s=np.random.randint(0,900)
# print(s)
# for j in range(s,s+1):
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     ax1.imshow(loaded_images[j])
#     ax2.imshow(loaded_masks[j])

## GAN Model

In [7]:
ngf= 64   # generator number of filters
ndf= 64   # discriminator number of filters
lrG=0.002 #0.0002
lrD=0.002
lamb=100  # loss weighting parameters
beta1=0.5 
beta2=0.999

In [8]:
print("number of samples",len(loaded_images))
print("image dim", loaded_images[0].shape)
print("mask  dim",loaded_masks[0].shape)

number of samples 1000
image dim (3, 512, 512)
mask  dim (3, 512, 512)


In [9]:
G = Generator(3, ngf, 3)
D = Discriminator(6, ndf, 1)
G.normal_weight_init(mean=0.0, std=0.02)
D.normal_weight_init(mean=0.0, std=0.02)

print(D)


Discriminator(
  (conv1): ConvBlock(
    (conv): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): ConvBlock(
    (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): ConvBlock(
    (conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, trac

In [10]:
print(G)

Generator(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): ConvBlock(
    (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): ConvBlock(
    (conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_ru

In [11]:
# to track the training epochs and runs

if not os.path.exists('id_log.txt'):
    with open('id_log.txt','w') as f:
        f.write('0')
with open('id_log.txt','r') as f:
    run_id = int(f.read())
    run_id+=1 

with open('id_log.txt','w') as f:
    f.write(str(run_id))

model_dir = 'GAN_model/'
save_dir='GAN_results/'
ex_dir=save_dir+str(run_id)+"/"


if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
if not os.path.exists(ex_dir):
    os.makedirs(ex_dir)
    
if not os.path.exists(model_dir):
    os.mkdir(model_dir)



In [12]:
# Loss functions
BCE_loss = torch.nn.BCELoss()#.cuda()
L1_loss = torch.nn.L1Loss()#.cuda()

# I used my laptop, cuda is an option
#G.cuda()
#D.cuda()


### GAN Training

In [14]:
# this is the last training trial, I used the whole dataset, but I sued 900:100 training:validation for parameters tuning


split_point=len(loaded_images)
X_train=np.array(loaded_images[:split_point])
Y_train=np.array(loaded_masks[:split_point])
im_train_names=np.array(pure_img_names[:split_point])
msk_train_names=np.array(pure_msk_names[:split_point])

# X_val=np.array(loaded_images[split_point:])
# Y_val=np.array(loaded_masks[split_point:])
# im_val_names=np.array(pure_img_names[split_point:])
# msk_val_names=np.array(pure_msk_names[split_point:])

print("train set:", len(Y_train),len(X_train))
# print("val   set:", len(Y_val),len(X_val))

train set: 1000 1000


In [15]:
# create a dataset class for the loader 
class Seg(Dataset):
    def __init__(self,X,Y,im_names,mask_names):

        self.len=X.shape[0]
        self.xdata=torch.from_numpy(X)
        self.ydata=torch.from_numpy(Y)
        
        self.xnames=im_names
        self.ynames=mask_names
        
    def __getitem__(self,index):
        return (self.xdata[index],self.ydata[index],self.xnames[index],self.ynames[index])
    
    def __len__(self):
        return(self.len)


In [16]:
# A function to save sample during training
# I created the function to save features from the middle of the netork
# the purpouse was to use them in creating graph later, however I stick to the GANs in this phase

def save_epoch_pairs(imgs_batch,
                     masks_batch,
                     gen_masks_batch, 
                     im_fn_batch,
                     msk_fn_batch,
                     im_dir_name,
                     msk_dir_name,
                     gen_msk_dir_name,
                     feat_dir_name):
    
   
    for i in range(len(imgs_batch)):
        
        im_fn =im_fn_batch[i]
        msk_fn=msk_fn_batch[i]
        

        
        gen_mask=gen_masks_batch[i].detach().numpy()#.permute(1,2,0).detach().numpy().astype(np.uint8)

        save_genmsk_fn = gen_msk_dir_name + msk_fn
        gen_mask_ = (((gen_mask -gen_mask.min()) * 255) / (gen_mask.max() - gen_mask.min())).transpose(1, 2, 0).astype(np.uint8)
        msk_gen = Image.fromarray(gen_mask_)
        msk_gen.save(save_genmsk_fn)
        #plt.show()
        plt.close()
        
        #save_msk_fn = msk_dir_name + msk_fn
        #msk_real = Image.fromarray(target_msk)
       # msk_real.save(save_msk_fn)
        #plt.show()
        #plt.close()
        
        
        # Saving features....
#         save_feat_fn = feat_dir_name + msk_fn[:-3]+"npz"
#         feat = features_batch[i].detach().numpy()
#         savez_compressed(save_feat_fn, feat)
        #torch.save(feat, save_feat_fn)


In [17]:

batch_size=8

train_seg=Seg(X_train,Y_train,im_train_names,msk_train_names)
train_loader=DataLoader(dataset=train_seg,batch_size=batch_size,shuffle=True)

# val_seg=Seg(X_val,Y_val,im_val_names,msk_val_names)
# val_loader=DataLoader(dataset=val_seg,batch_size=2,shuffle=True)



In [19]:

# Optimizers
G_optimizer = torch.optim.Adam(G.parameters(), lr=lrG, betas=(beta1, beta2))
D_optimizer = torch.optim.Adam(D.parameters(), lr=lrD, betas=(beta1, beta2))

In [20]:
# Training GAN
D_avg_losses = []
G_avg_losses = []
num_epochs=10
step = 0
for epoch in range(num_epochs):
    
    epoch_dir=ex_dir+str(epoch)+"/"
    if not os.path.exists(epoch_dir):
        os.mkdir(epoch_dir)


    epoch_im_dir=epoch_dir+"images/"


    epoch_msk_dir=epoch_dir+"masks/"


    epoch_genmsk_dir=epoch_dir+"gen_masks/"
    if not os.path.exists(epoch_genmsk_dir):
        os.mkdir(epoch_genmsk_dir)


    epoch_feat_dir=epoch_dir+"features/"
    if not os.path.exists(epoch_feat_dir):
        os.mkdir(epoch_feat_dir)

    D_losses = []
    G_losses = []

    # training
    for i, (input, target, im_file_names, msk_file_names) in enumerate(train_loader):

        # input & target image data
        x_ = Variable(input)#.permute(0,3,1,2)#Variable(input.cuda())
        y_ = Variable(target)#.permute(0,3,1,2)#Variable(target.cuda())

        x_ = x_.float()#Variable(input.cuda())
        y_ = y_.float()#Variable(target.cuda())

        # Train discriminator with real data
        D_real_decision = D(x_, y_).squeeze()
        real_ = Variable(torch.ones(D_real_decision.size())) #Variable(torch.ones(D_real_decision.size()).cuda())
        D_real_loss = BCE_loss(D_real_decision, real_)

        # Train discriminator with fake data
        gen_image, _ = G(x_)
        D_fake_decision = D(x_, gen_image).squeeze()
        fake_ = Variable(torch.zeros(D_fake_decision.size()))#Variable(torch.zeros(D_fake_decision.size()).cuda())
        D_fake_loss = BCE_loss(D_fake_decision, fake_)

        # Back propagation
        D_loss = (D_real_loss + D_fake_loss) * 0.5
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train generator
#         gen_image, features = G(x_)
        gen_image, _ = G(x_)
        D_fake_decision = D(x_, gen_image).squeeze()
        G_fake_loss = BCE_loss(D_fake_decision, real_)

        # L1 loss
        l1_loss = lamb * L1_loss(gen_image, y_)

        # Back propagation
        G_loss = G_fake_loss + l1_loss
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # loss values
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())
        
        #features_proj=F.interpolate(features,scale_factor=(2,2), mode='nearest')

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, num_epochs, i+1, len(train_loader), D_loss.item(), G_loss.item()))
        

#         if epoch>=0:
#             save_epoch_pairs(x_,
#                              y_,
#                              gen_image,
#                              features_proj,
#                              im_file_names,
#                              msk_file_names,
#                              epoch_im_dir,
#                              epoch_msk_dir,
#                              epoch_genmsk_dir,
#                              epoch_feat_dir)

        
        if epoch>=4:
            save_epoch_pairs(x_,
                             y_,
                             gen_image,
                             im_file_names,
                             msk_file_names,
                             epoch_im_dir,
                             epoch_msk_dir,
                             epoch_genmsk_dir,
                             epoch_feat_dir)

        
#         save_epoch_pairs(imgs_batch,
#                      masks_batch,
#                      gen_masks_batch, 
#                      im_fn_batch,
#                      msk_fn_batch,
#                      im_dir_name,
#                      msk_dir_name,
#                      gen_msk_dir_name):

    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    # avg loss values for plot
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    # Show result for test image
    
#     val_input, val_target, val_img_names, val_msk_names = val_loader.__iter__().__next__()

#     gen_image = G(Variable(test_input))
#     gen_image = gen_image.cpu().data
#     plot_test_result(test_input, test_target, gen_image, epoch, save=True, ex_dir=ex_dir)
    
    save_model_fn=ex_dir+"Seg_GAN.pth"
        
    torch.save({"g_state_dict":G.state_dict(),
                "d_state_dict":D.state_dict(),
                "g_opt_dict":G_optimizer.state_dict(),
                "d_opt_dict":G_optimizer.state_dict(),
                       },
                       save_model_fn
                      )

Epoch [1/10], Step [1/125], D_loss: 0.8207, G_loss: 101.8363
Epoch [1/10], Step [2/125], D_loss: 3.5759, G_loss: 86.8924
Epoch [1/10], Step [3/125], D_loss: 4.8366, G_loss: 68.9630
Epoch [1/10], Step [4/125], D_loss: 1.8384, G_loss: 66.5402
Epoch [1/10], Step [5/125], D_loss: 1.4256, G_loss: 49.0444
Epoch [1/10], Step [6/125], D_loss: 1.3846, G_loss: 47.7315
Epoch [1/10], Step [7/125], D_loss: 0.8463, G_loss: 37.9742
Epoch [1/10], Step [8/125], D_loss: 0.9875, G_loss: 40.8745
Epoch [1/10], Step [9/125], D_loss: 0.7572, G_loss: 28.5668
Epoch [1/10], Step [10/125], D_loss: 0.9490, G_loss: 28.2638
Epoch [1/10], Step [11/125], D_loss: 0.6944, G_loss: 21.5612
Epoch [1/10], Step [12/125], D_loss: 0.7441, G_loss: 26.4606
Epoch [1/10], Step [13/125], D_loss: 0.7678, G_loss: 31.3790
Epoch [1/10], Step [14/125], D_loss: 0.7797, G_loss: 26.7046
Epoch [1/10], Step [15/125], D_loss: 1.0174, G_loss: 36.2731
Epoch [1/10], Step [16/125], D_loss: 0.9700, G_loss: 20.4294
Epoch [1/10], Step [17/125], D_l

Epoch [2/10], Step [11/125], D_loss: 0.9730, G_loss: 41.7404
Epoch [2/10], Step [12/125], D_loss: 0.9440, G_loss: 49.5962
Epoch [2/10], Step [13/125], D_loss: 0.5832, G_loss: 32.5513
Epoch [2/10], Step [14/125], D_loss: 1.6083, G_loss: 36.4134
Epoch [2/10], Step [15/125], D_loss: 1.4151, G_loss: 20.0023
Epoch [2/10], Step [16/125], D_loss: 1.2524, G_loss: 23.5814
Epoch [2/10], Step [17/125], D_loss: 1.6765, G_loss: 25.1462
Epoch [2/10], Step [18/125], D_loss: 0.7223, G_loss: 21.3001
Epoch [2/10], Step [19/125], D_loss: 1.3260, G_loss: 14.3448
Epoch [2/10], Step [20/125], D_loss: 0.5670, G_loss: 48.4337
Epoch [2/10], Step [21/125], D_loss: 0.3567, G_loss: 29.0761
Epoch [2/10], Step [22/125], D_loss: 0.2613, G_loss: 19.9000
Epoch [2/10], Step [23/125], D_loss: 0.3194, G_loss: 22.2057
Epoch [2/10], Step [24/125], D_loss: 0.1638, G_loss: 45.4899
Epoch [2/10], Step [25/125], D_loss: 0.2162, G_loss: 34.7242
Epoch [2/10], Step [26/125], D_loss: 0.2472, G_loss: 34.3454
Epoch [2/10], Step [27/1

Epoch [3/10], Step [21/125], D_loss: 0.3077, G_loss: 30.7647
Epoch [3/10], Step [22/125], D_loss: 0.3545, G_loss: 40.0739
Epoch [3/10], Step [23/125], D_loss: 0.0820, G_loss: 37.0724
Epoch [3/10], Step [24/125], D_loss: 0.1499, G_loss: 37.8397
Epoch [3/10], Step [25/125], D_loss: 0.7976, G_loss: 27.2374
Epoch [3/10], Step [26/125], D_loss: 1.1544, G_loss: 45.0643
Epoch [3/10], Step [27/125], D_loss: 0.5131, G_loss: 32.4753
Epoch [3/10], Step [28/125], D_loss: 0.9170, G_loss: 26.1090
Epoch [3/10], Step [29/125], D_loss: 1.1206, G_loss: 31.3289
Epoch [3/10], Step [30/125], D_loss: 0.2543, G_loss: 39.9177
Epoch [3/10], Step [31/125], D_loss: 1.6109, G_loss: 32.5239
Epoch [3/10], Step [32/125], D_loss: 0.6603, G_loss: 24.4811
Epoch [3/10], Step [33/125], D_loss: 0.6432, G_loss: 39.5004
Epoch [3/10], Step [34/125], D_loss: 0.4362, G_loss: 41.8999
Epoch [3/10], Step [35/125], D_loss: 0.7485, G_loss: 26.6946
Epoch [3/10], Step [36/125], D_loss: 0.3032, G_loss: 34.8607
Epoch [3/10], Step [37/1

Epoch [4/10], Step [31/125], D_loss: 0.6107, G_loss: 36.6452
Epoch [4/10], Step [32/125], D_loss: 0.7256, G_loss: 31.3806
Epoch [4/10], Step [33/125], D_loss: 0.9987, G_loss: 35.4280
Epoch [4/10], Step [34/125], D_loss: 0.5276, G_loss: 36.5873
Epoch [4/10], Step [35/125], D_loss: 0.2949, G_loss: 27.8499
Epoch [4/10], Step [36/125], D_loss: 0.3605, G_loss: 30.9728
Epoch [4/10], Step [37/125], D_loss: 0.4339, G_loss: 33.9018
Epoch [4/10], Step [38/125], D_loss: 1.1758, G_loss: 28.6264
Epoch [4/10], Step [39/125], D_loss: 0.2992, G_loss: 53.7267
Epoch [4/10], Step [40/125], D_loss: 0.2223, G_loss: 36.7450
Epoch [4/10], Step [41/125], D_loss: 0.3536, G_loss: 25.6838
Epoch [4/10], Step [42/125], D_loss: 0.3954, G_loss: 35.9829
Epoch [4/10], Step [43/125], D_loss: 0.6840, G_loss: 29.9420
Epoch [4/10], Step [44/125], D_loss: 0.4196, G_loss: 41.8854
Epoch [4/10], Step [45/125], D_loss: 0.4072, G_loss: 35.6728
Epoch [4/10], Step [46/125], D_loss: 0.8024, G_loss: 33.0632
Epoch [4/10], Step [47/1

Epoch [5/10], Step [41/125], D_loss: 0.4216, G_loss: 29.9425
Epoch [5/10], Step [42/125], D_loss: 0.3184, G_loss: 37.0852
Epoch [5/10], Step [43/125], D_loss: 0.5411, G_loss: 37.2782
Epoch [5/10], Step [44/125], D_loss: 0.2218, G_loss: 22.0234
Epoch [5/10], Step [45/125], D_loss: 1.2125, G_loss: 24.2711
Epoch [5/10], Step [46/125], D_loss: 0.6710, G_loss: 19.7378
Epoch [5/10], Step [47/125], D_loss: 0.1366, G_loss: 31.9146
Epoch [5/10], Step [48/125], D_loss: 0.3660, G_loss: 30.0545
Epoch [5/10], Step [49/125], D_loss: 0.0708, G_loss: 35.0090
Epoch [5/10], Step [50/125], D_loss: 0.3336, G_loss: 32.2116
Epoch [5/10], Step [51/125], D_loss: 0.2150, G_loss: 45.4243
Epoch [5/10], Step [52/125], D_loss: 0.3996, G_loss: 31.6208
Epoch [5/10], Step [53/125], D_loss: 0.4748, G_loss: 27.8607
Epoch [5/10], Step [54/125], D_loss: 0.7630, G_loss: 35.1560
Epoch [5/10], Step [55/125], D_loss: 2.7549, G_loss: 25.4806
Epoch [5/10], Step [56/125], D_loss: 0.9264, G_loss: 36.8633
Epoch [5/10], Step [57/1

Epoch [6/10], Step [51/125], D_loss: 0.5120, G_loss: 21.5194
Epoch [6/10], Step [52/125], D_loss: 0.2179, G_loss: 22.3535
Epoch [6/10], Step [53/125], D_loss: 0.5720, G_loss: 30.7626
Epoch [6/10], Step [54/125], D_loss: 0.4758, G_loss: 25.3999
Epoch [6/10], Step [55/125], D_loss: 0.2623, G_loss: 32.5829
Epoch [6/10], Step [56/125], D_loss: 1.9295, G_loss: 26.9746
Epoch [6/10], Step [57/125], D_loss: 0.2530, G_loss: 33.9490
Epoch [6/10], Step [58/125], D_loss: 0.8434, G_loss: 20.3605
Epoch [6/10], Step [59/125], D_loss: 0.6464, G_loss: 23.2717
Epoch [6/10], Step [60/125], D_loss: 0.3495, G_loss: 25.6460
Epoch [6/10], Step [61/125], D_loss: 0.2604, G_loss: 27.1028
Epoch [6/10], Step [62/125], D_loss: 0.2350, G_loss: 43.8890
Epoch [6/10], Step [63/125], D_loss: 0.5963, G_loss: 22.1660
Epoch [6/10], Step [64/125], D_loss: 0.1394, G_loss: 45.5188
Epoch [6/10], Step [65/125], D_loss: 0.9731, G_loss: 25.8334
Epoch [6/10], Step [66/125], D_loss: 1.2677, G_loss: 16.4998
Epoch [6/10], Step [67/1

Epoch [7/10], Step [61/125], D_loss: 1.3558, G_loss: 14.9831
Epoch [7/10], Step [62/125], D_loss: 0.4619, G_loss: 23.3790
Epoch [7/10], Step [63/125], D_loss: 1.1183, G_loss: 20.6010
Epoch [7/10], Step [64/125], D_loss: 0.6076, G_loss: 26.5957
Epoch [7/10], Step [65/125], D_loss: 0.5092, G_loss: 27.4717
Epoch [7/10], Step [66/125], D_loss: 1.2486, G_loss: 23.8506
Epoch [7/10], Step [67/125], D_loss: 0.4606, G_loss: 39.6733
Epoch [7/10], Step [68/125], D_loss: 0.3399, G_loss: 37.0134
Epoch [7/10], Step [69/125], D_loss: 0.7382, G_loss: 26.5749
Epoch [7/10], Step [70/125], D_loss: 0.8012, G_loss: 29.6810
Epoch [7/10], Step [71/125], D_loss: 0.1606, G_loss: 39.9735
Epoch [7/10], Step [72/125], D_loss: 0.6483, G_loss: 19.1168
Epoch [7/10], Step [73/125], D_loss: 0.1383, G_loss: 35.5513
Epoch [7/10], Step [74/125], D_loss: 0.7700, G_loss: 17.7228
Epoch [7/10], Step [75/125], D_loss: 1.7778, G_loss: 32.9964
Epoch [7/10], Step [76/125], D_loss: 0.6413, G_loss: 22.1895
Epoch [7/10], Step [77/1

Epoch [8/10], Step [71/125], D_loss: 0.5967, G_loss: 21.4030
Epoch [8/10], Step [72/125], D_loss: 0.2719, G_loss: 31.7750
Epoch [8/10], Step [73/125], D_loss: 0.3901, G_loss: 17.6869
Epoch [8/10], Step [74/125], D_loss: 0.2912, G_loss: 20.3583
Epoch [8/10], Step [75/125], D_loss: 0.9322, G_loss: 14.9074
Epoch [8/10], Step [76/125], D_loss: 0.6532, G_loss: 29.7455
Epoch [8/10], Step [77/125], D_loss: 0.3779, G_loss: 26.9162
Epoch [8/10], Step [78/125], D_loss: 0.3715, G_loss: 30.9187
Epoch [8/10], Step [79/125], D_loss: 0.3027, G_loss: 27.2999
Epoch [8/10], Step [80/125], D_loss: 0.4594, G_loss: 26.9159
Epoch [8/10], Step [81/125], D_loss: 0.4329, G_loss: 26.3081
Epoch [8/10], Step [82/125], D_loss: 1.0024, G_loss: 23.6033
Epoch [8/10], Step [83/125], D_loss: 0.8066, G_loss: 35.8305
Epoch [8/10], Step [84/125], D_loss: 0.1740, G_loss: 23.8443
Epoch [8/10], Step [85/125], D_loss: 0.3339, G_loss: 25.4874
Epoch [8/10], Step [86/125], D_loss: 0.7342, G_loss: 24.2318
Epoch [8/10], Step [87/1

Epoch [9/10], Step [81/125], D_loss: 0.3618, G_loss: 27.9421
Epoch [9/10], Step [82/125], D_loss: 0.2587, G_loss: 23.7022
Epoch [9/10], Step [83/125], D_loss: 0.5295, G_loss: 25.9255
Epoch [9/10], Step [84/125], D_loss: 1.0343, G_loss: 18.1159
Epoch [9/10], Step [85/125], D_loss: 0.1790, G_loss: 37.4511
Epoch [9/10], Step [86/125], D_loss: 0.1388, G_loss: 28.4284
Epoch [9/10], Step [87/125], D_loss: 0.4523, G_loss: 28.4875
Epoch [9/10], Step [88/125], D_loss: 0.2183, G_loss: 23.5357
Epoch [9/10], Step [89/125], D_loss: 0.2894, G_loss: 26.6540
Epoch [9/10], Step [90/125], D_loss: 0.2992, G_loss: 36.6135
Epoch [9/10], Step [91/125], D_loss: 0.1538, G_loss: 24.4400
Epoch [9/10], Step [92/125], D_loss: 0.6560, G_loss: 21.0161
Epoch [9/10], Step [93/125], D_loss: 0.2256, G_loss: 30.9289
Epoch [9/10], Step [94/125], D_loss: 0.9592, G_loss: 31.4289
Epoch [9/10], Step [95/125], D_loss: 1.4048, G_loss: 22.2356
Epoch [9/10], Step [96/125], D_loss: 0.2143, G_loss: 41.4937
Epoch [9/10], Step [97/1

Epoch [10/10], Step [89/125], D_loss: 0.1808, G_loss: 30.3701
Epoch [10/10], Step [90/125], D_loss: 0.3344, G_loss: 18.4775
Epoch [10/10], Step [91/125], D_loss: 0.3086, G_loss: 24.3789
Epoch [10/10], Step [92/125], D_loss: 0.3960, G_loss: 25.7738
Epoch [10/10], Step [93/125], D_loss: 0.0990, G_loss: 21.0158
Epoch [10/10], Step [94/125], D_loss: 0.5896, G_loss: 21.1525
Epoch [10/10], Step [95/125], D_loss: 0.8381, G_loss: 30.9998
Epoch [10/10], Step [96/125], D_loss: 0.3467, G_loss: 15.7210
Epoch [10/10], Step [97/125], D_loss: 0.3405, G_loss: 46.2921
Epoch [10/10], Step [98/125], D_loss: 1.7054, G_loss: 20.7359
Epoch [10/10], Step [99/125], D_loss: 0.4328, G_loss: 22.9821
Epoch [10/10], Step [100/125], D_loss: 0.3469, G_loss: 19.4675
Epoch [10/10], Step [101/125], D_loss: 1.1468, G_loss: 11.0787
Epoch [10/10], Step [102/125], D_loss: 0.2926, G_loss: 24.7575
Epoch [10/10], Step [103/125], D_loss: 0.4013, G_loss: 28.0345
Epoch [10/10], Step [104/125], D_loss: 0.2574, G_loss: 20.8618
Epo

In [ ]:
    
# save_model_fn=ex_dir+"Seg_GAN.pth"

# torch.save({"g_state_dict":G.state_dict(),
#             "d_state_dict":D.state_dict(),
#             "g_opt_dict":G_optimizer.state_dict(),
#             "d_opt_dict":G_optimizer.state_dict(),
#                    },
#                    save_model_fn
#                   )